In [3]:
import numpy as np
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib as plt
from sklearn.metrics.pairwise import cosine_similarity
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')
movies =movies.merge(credits, on = 'title')
movies = movies[['movie_id','title','genres','overview','release_date','cast','crew','keywords']]
movies.dropna(inplace = True)
movies['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [4]:
def convert(str):
    L= []
    counter = 0
    for i in ast.literal_eval(str):
        if counter != 3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L
def fetch_director(obj):
    L= []
    # counter = 0
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L

In [5]:
movies['genres']=movies['genres'].apply(convert)
# movies['keywords']=movies['keywords'].apply(convert)
# movies['cast']

In [6]:
movies['keywords']=movies['keywords'].apply(convert)

In [7]:
movies['cast'] = movies['cast'].apply(convert)

In [8]:
movies['crew'] =movies['crew'].apply(fetch_director)

In [9]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [10]:
movies['genres'] =movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] =movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] =movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies['overview'] =movies['overview'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] =movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [11]:
movies['tags'] = movies['overview']+movies['genres']+movies['cast']+movies['crew']+movies['keywords']

In [12]:
df = movies[['movie_id','title','tags']]

In [13]:
df['tags'] = df['tags'].apply(lambda x:" ".join(x))

C:\Users\My Desktop\AppData\Local\Temp\ipykernel_6092\4267108141.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x:" ".join(x))


In [14]:
df['tags'] = df['tags'].apply(lambda x:x.lower()) 

C:\Users\My Desktop\AppData\Local\Temp\ipykernel_6092\1119052810.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x:x.lower())


In [15]:
df['tags']

0       in the 22nd century, a paraplegic marine is di...
1       captain barbossa, long believed to be dead, ha...
2       a cryptic message from bond’s past sends him o...
3       following the death of district attorney harve...
4       john carter is a war-weary, former military ca...
                              ...                        
4804    el mariachi just wants to play his guitar and ...
4805    a newlywed couple's honeymoon is upended by th...
4806    "signed, sealed, delivered" introduces a dedic...
4807    when ambitious new york attorney sam is sent t...
4808    ever since the second grade when he first saw ...
Name: tags, Length: 4805, dtype: object

In [16]:
cv = CountVectorizer(max_features=5000, stop_words='english')

In [17]:
vector = cv.fit_transform(df['tags']).toarray()

In [18]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [19]:
vector.shape

(4805, 5000)

In [20]:
similarity = cosine_similarity(vector)

In [21]:
def recommend(movie):
    index = df[df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(df.iloc[i[0]].title)

In [22]:
recommend('Star Trek: Insurrection')

Star Trek: First Contact
Star Trek: Nemesis
Star Trek: Generations
Star Trek IV: The Voyage Home
Star Trek III: The Search for Spock


In [23]:
import pickle
pickle.dump(df,open('movie_list.pk1','wb'))
pickle.dump(similarity,open('similarity.pk1','wb'))